In [2]:
# concept/concept.py

import vertexai
from vertexai.generative_models import (
    Content,
    FunctionDeclaration,
    GenerationConfig,
    GenerativeModel,
    Part,
    Tool,
)
import json

# Initialize Vertex AI with your project ID and location
vertexai.init(project="mm-cia-dev", location="europe-west1")  # Replace with your project ID and location

In [5]:
# Function to get the name of a sibling
def get_sibling(sibling: str) -> dict:
    """
    Returns the name of a sibling based on the type.
    """
    if sibling.lower() == "sister":
        return {"result": "Mary"}
    elif sibling.lower() == "brother":
        return {"result": "John"}
    else:
        return {"result": "Unknown"}

get_sibling_func = FunctionDeclaration(
    name="get_sibling",
    description="Returns the name of a sibling based on the type.",
    parameters={
        "type": "object",
        "properties": {
            "sibling": {
                "type": "string",
                "description": "Type of sibling (e.g., sister, brother)"
            }
        },
        "required": ["sibling"]
    }
)

sibling_tool = Tool(
    function_declarations=[get_sibling_func],
)



In [5]:

# Define the tool that includes the function declaration
# Define the user's prompt as a Content object
user_prompt_content = "Can you give me the name of my sister?"


# Initialize the Gemini model with the specified tools
model = GenerativeModel(
    "gemini-1.5-flash-002",  # Replace with your desired Gemini model ID
    system_instruction="You are a helpful assistant that can answer questions and help with tasks. You have tools to get the name of a sibling. If the user ask for a name you can use a tool to get either the name of the brother or the sister, depending on what the user ask for.",
    generation_config=GenerationConfig(temperature=0.5),
    tools=[sibling_tool],
)

# Generate content with the user prompt
response = model.generate_content(
    user_prompt_content,
)
print("Response:")
print(response)


# Extract the function call from the model's response
function_call = response.candidates[0].function_calls[0]

print("Function Call Detected:")
print(json.dumps(function_call, indent=4))



NameError: name 'sibling_tool' is not defined

In [7]:
model = GenerativeModel(
    "gemini-1.5-flash-002",  # Replace with your desired Gemini model ID
    system_instruction="You are an expert in transforming the content of webpages into structured markdown. You just receive the result from curl and you need to transform it into markdown.",
    generation_config=GenerationConfig(temperature=0.5),
)
# Curl "https://googlecloudgeminihackathon.devpost.com/?ref_feature=challenge&ref_medium=discover"
import requests

# Fetch content from the URL
url = "https://googlecloudgeminihackathon.devpost.com/?ref_feature=challenge&ref_medium=discover"
page_content = requests.get(url).text

import weave
# Generate content with the user prompt
weave.init("castor-concept")

@weave.op()
def main(page_content: str):
    response = model.generate_content(
        page_content)
    return response

response = main(page_content)
print("Response:")
print(response.text)

Logged in as Weights & Biases user: 100386322.
View Weave data at https://wandb.ai/100386322/castor-concept/weave
🍩 https://wandb.ai/100386322/castor-concept/r/call/0192c5a6-89bd-75e1-b99a-a3a3ff6a29e1
Response:
# Google Cloud Gemini Hackathon

**This hackathon is open to registrants from Europe, Middle East and Africa (specific countries).**

## Overview

<p>Welcome to the Google Cloud Gemini Hackathon! Join us for an exciting hackathon where creativity meets cutting-edge technology. We invite developers, innovators, and tech enthusiasts to come together and build groundbreaking applications using the Gemini API, in either Google AI Studio or Vertex AI environment. This is your chance to create solutions that not only showcase your skills but also address some of the most pressing challenges in the modern world.</p>


**Join Hackathon:** [https://googlecloudgeminihackathon.devpost.com/register?flow%5Bdata%5D%5Bchallenge_id%5D=22241&amp;flow%5Bname%5D=register_for_challenge](https://go

In [ ]:


# Execute the function based on the model's function call
if function_call.name == function_name:
    sibling_type = function_call.args["sibling"]
    function_result = get_sibling(sibling_type)
    print("\nFunction Execution Result:")
    print(json.dumps(function_result, indent=4))

    # Prepare the function response as Content
    function_response_content = Content(
        role="function",
        name=function_name,
        parts=[
            Part.from_function_response(
                name=function_name,
                response=function_result
            )
        ],
    )

    # Generate the final model response with the function result
    final_response = model.generate_content(
        [
            user_prompt_content,          # User prompt
            response.candidates[0].content,  # Function call response
            function_response_content     # Function execution result
        ],
        tools=[sibling_tool],
    )

    print("\nFinal Model Response:")
    print(final_response.text)